In [2]:
import numpy as np

In [3]:
sign_function = np.vectorize(lambda x : 1 if x >= 0 else -1)

In [12]:
class Hopfield :
    def __init__ (self,n_cells, memories=[]):
        self.n_cells = n_cells
        self.weights = np.mat(np.zeros((n_cells,n_cells)))
        self.memories = memories
        if len(self.memories) > 0:
            for m in memories:
                self.addMemory(m)
            self.finish()
        
    def addMemory(self, memory):
        self.weights = self.weights + (memory.T * memory - np.mat(np.eye(self.n_cells)))
        
    def finish(self):
        self.weights = self.weights * (1/float(self.n_cells))
        
    def energy(self, state):
        return np.asscalar(-1/float(2) * state * self.weights * state.T)
        
    def findMinimum(self,state):
        actual_state = state
        equals = False
        while not equals :
            print self.energy(actual_state)
            next_state = np.mat(np.zeros(self.n_cells))
            for i in range(0,self.n_cells):
                sign = np.asscalar(sign_function(self.weights[i,:] * state.T))
                next_state[:i] = sign
            equals = np.all(np.equal(actual_state,next_state))
            actual_state = next_state
        return actual_state

In [13]:
N = 1000
def hopfield_stable(hn):
    return np.all([np.all(np.equal(memory,hn.findMinimum(memory))) for memory in hn.memories])
stable = True
memories = []
while stable:
    memories.append(sign_function(np.mat(np.random.uniform(-1,1,N))))
    hn = Hopfield(N,memories)
    stable = hopfield_stable(hn)
print len(memories)

-499.5
0.3
1


In [29]:
N = 3
memory = sign_function(np.mat(np.random.uniform(-1,1,N)))
hn = Hopfield(N)
hn.addMemory(memory)
print memory
print hn.findMinimum(memory)
print hn.weights

[[-1  1 -1]]
-3.0
1.0
[[-1. -1. -1.]]
[[ 0. -1.  1.]
 [-1.  0. -1.]
 [ 1. -1.  0.]]


In [15]:
hn = Hopfield(7)
hn.addMemory(np.mat([1,1,1,1,1,1,1]))
print hn.findMinimum(np.mat([1,-1,-1,1,1,1,1]))
print hn.findMinimum(np.mat([-1,-1,-1,1,-1,-1,-1]))

-1.0
-21.0
[[ 1.  1.  1.  1.  1.  1.  1.]]
-9.0
-21.0
[[-1. -1. -1. -1. -1. -1. -1.]]


In [16]:
f = np.vectorize(lambda x,y: x*x +y)

In [78]:
f([1,2,3],2)

array([ 3,  6, 11])

In [127]:
eye = np.eye(3)
np.matrix(eye)+np.matrix(eye) * 1/float(2)

matrix([[ 1.5,  0. ,  0. ],
        [ 0. ,  1.5,  0. ],
        [ 0. ,  0. ,  1.5]])

In [92]:
M = np.matrix(np.zeros((3,3)))
v = np.matrix([1,2,3])
M + (v.T*v) - np.matrix(np.eye(3))

matrix([[ 0.,  2.,  3.],
        [ 2.,  3.,  6.],
        [ 3.,  6.,  8.]])

In [18]:
v = np.mat([1,2,3])
v1 = np.mat([1,3])

In [19]:
np.all(np.equal(v,v1))

ValueError: operands could not be broadcast together with shapes (1,3) (1,2) 